In [2]:
import os
import warnings
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor

warnings.filterwarnings('ignore')

In [3]:
# 데이터 불러오기
total_data_path = os.path.join('data','preprocessed_data.csv')
y_path = os.path.join('data','label.csv')
sub_path = os.path.join('data','sample_submission.csv')

total_data = pd.read_csv(total_data_path)
y = pd.read_csv(y_path)
submission = pd.read_csv(sub_path)

In [4]:
len(y)

15035

In [5]:
# train test 분리
train = total_data.loc[:len(y)-1]
test = total_data.loc[len(y):].reset_index(drop=True)

In [6]:
# 원본 보존
train2 = train.copy()
test2 = test.copy()

In [7]:
# 원 핫 사용할 컬럼
cols = ['bedrooms','bathrooms','floors','waterfront','view','condition','grade']

In [8]:
train2 = pd.get_dummies(train2, columns=cols)
test2 = pd.get_dummies(test2, columns=cols)

In [9]:
set(train2.columns)  - set(test2.columns)

{'bathrooms_7.5', 'bathrooms_7.75', 'bathrooms_8.0', 'grade_1'}

In [10]:
set(test2.columns)  - set(train2.columns)

{'bathrooms_6.5', 'bedrooms_11', 'bedrooms_33'}

In [11]:
# 컬럼 맞춰주기
train2['bathrooms_6.5'] = np.zeros(train2.shape[0],)
train2['bedrooms_11'] = np.zeros(train2.shape[0],)
train2['bedrooms_33'] = np.zeros(train2.shape[0],)

In [12]:
test2['bathrooms_7.5'] = np.zeros(test2.shape[0],)
test2['bathrooms_7.75'] = np.zeros(test2.shape[0],)
test2['bathrooms_8.0'] = np.zeros(test2.shape[0],)
test2['grade_1'] = np.zeros(test2.shape[0],)

# Model

In [13]:
seed = 123

In [14]:
params ={
    "random_state" : seed,
    "n_estimators" : 400
}

## 랜덤포레스트로 간단한 모델만들기
\+ bagging을 하여 30개 모델을 앙상블하여 구현함  
\+ AIFFEL 1차 hackaton 1등 당시 썼던 테크닉  
\+ lightgbm같은 모델을 사용하고 파라미터 튜닝도하면 성능도 더 잘나옴

In [15]:
# Ensemble 
bagging_predict = [] # 예측 값 array 저장을 위한 리스트
for _ in range(30): # iteration 30회 - 30회는 임의로 정한 숫자
    data_index = [idx  for idx in range(train2.shape[0])] # 전체 데이터의 인덱스를 추출한 리스트
    random_index = np.random.choice(data_index, train2.shape[0]) # 복원추출 적용
    rf = RandomForestRegressor(**params)
    rf.fit(train2.iloc[random_index,], y.iloc[random_index,])
    
    predict = rf.predict(test2) # prediction
    bagging_predict.append(predict) # 예측 값 array 저장

In [18]:
prediction = [] # 예측 값 리스트
for idx2 in range(test2.shape[0]): # test의 index 만큼 iteration
    temp = []
    # 각 array line by line 으로 평균 내어 prediction에 저장
    for idx in range(len(bagging_predict)): 
        temp.append(bagging_predict[idx][idx2])
    prediction.append(np.mean(temp))

In [21]:
submission["price"] = prediction

In [22]:
submission.to_csv("submission_baseline.csv", index=False)